In [ ]:
#import necessary libraries
import pandas as pd
import numpy as np
import panel as pn
import hvplot.xarray
import hvplot.pandas
import datetime as dt
from datetime import datetime
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import cycle
import mysql.connector
from csv import QUOTE_NONNUMERIC, QUOTE_NONE
import dataframe_image as df_image
from holoviews import opts

pn.extension('tabulator')
pn.extension()
from sqlalchemy import create_engine

<div class="alert alert-block alert-danger">
<b>Bis Zelle 9 überspringen und ab da den Code ausführen, wenn keine Verbindung zur MariaDB besteht.</b>
</div>

## Datensätze aus der MySQL (MariaDB) Datenbank auslesen

<div class="alert alert-block alert-info">
<b>Hilfsfunktion</b>
</div>

In [ ]:
def get_dataframe(table_name):
    con = mysql.connector.connect(host="localhost", database='moodle', user="root", passwd="",use_pure=True)

    cursor = con.cursor()
    query = "SELECT * FROM " + table_name
    cursor.execute(query)

    df = pd.DataFrame(cursor.fetchall())
    df.columns = cursor.column_names

    cursor.close()
    con.close()
    
    return df

<div class="alert alert-block alert-info">
<b>Datensätze aus der Datenbank</b>
</div>

In [ ]:
user = get_dataframe('mdl_user')
quiz = get_dataframe('mdl_quiz')
quiz_attempts = get_dataframe('mdl_quiz_attempts')
quiz_slots = get_dataframe('mdl_quiz_slots')
quiz_grades = get_dataframe('mdl_quiz_grades')
quiz_feedback = get_dataframe('mdl_quiz_feedback')
question = get_dataframe('mdl_question')
question_attempts = get_dataframe('mdl_question_attempts')
question_usages = get_dataframe('mdl_question_usages')
course = get_dataframe('mdl_course')
course_modules = get_dataframe('mdl_course_modules')
course_completion = get_dataframe('mdl_course_completions')
user_enrolments = get_dataframe('mdl_user_enrolments')
enrol = get_dataframe('mdl_enrol')
course_sections = get_dataframe('mdl_course_sections')

## Eigenen Datensatz für das Dashboard aufbauen

<div class="alert alert-block alert-info">
<b>Notwendige Daten aus Datenbank filtern</b>
</div>

In [ ]:
user_data = {'Name': user['firstname'] + ' ' + user['lastname'],
                      'SuS ID': user['id']
                     }

quiz_data = {'Aufgabe': quiz['name'],
             'Quiz ID': quiz['id'],
             'Maximale Punktzahl': quiz['sumgrades'],
             'Maximale Note in %': quiz['grade']
            }

quiz_attempts_data = {'SuS ID': quiz_attempts['userid'],
                      'Quiz ID': quiz_attempts['quiz'],
                      'Fragennutzung ID': quiz_attempts['uniqueid'],
                      'Quiz Startzeit': quiz_attempts['timestart'],
                      'Quiz Abgabezeit': quiz_attempts['timefinish'],
                      'Punktzahl': quiz_attempts['sumgrades'],
                      'Anzahl Versuche': quiz_attempts['attempt']
                     }

quiz_grade_data = {'Beste Punktzahl in %': quiz_grades['grade'],
                   'SuS ID': quiz_grades['userid'],
                   'Quiz ID': quiz_grades['quiz']
                  }

user_enrolments_data = {'Anmelde ID': user_enrolments['enrolid'],
                        'SuS ID': user_enrolments['userid']
                       }

enrol_data = {'Anmelde ID': enrol['id'],
              'Klassen ID': enrol['courseid']
             }

course_data = {'Klasse': course['fullname'],
               'Klassen ID': course['id']
              }

course_sections_data = {'Klassen ID': course_sections['course'],
                        'Fach': course_sections['name']
                       }

question_attempts_data = {'Frage Versuch ID': question_attempts['id'],
                          'Fragennutzung ID': question_attempts['questionusageid'],
                          'Slot': question_attempts['slot'],
                          'Frage ID': question_attempts['questionid'],
                          'Frage Text': question_attempts['questionsummary'],
                          'Antwort': question_attempts['responsesummary']
                         }

question_data = {'Frage ID': question['id'],
                 'Frage Name': question['name']
                }

<div class="alert alert-block alert-info">
<b>Notwendige Daten zusammenfügen für einen großen Datensatzn</b>
</div>

In [ ]:
df_user = pd.DataFrame(user_data)

df_quiz_attempts = pd.DataFrame(quiz_attempts_data)
df_quiz_attempts['Quiz Startzeit'] = pd.to_datetime(df_quiz_attempts['Quiz Startzeit'], unit = 's')
df_quiz_attempts['Quiz Abgabezeit'] = pd.to_datetime(df_quiz_attempts['Quiz Abgabezeit'], unit = 's')
df_quiz_attempts['Quiz Bearbeitungsdauer'] = df_quiz_attempts['Quiz Abgabezeit'] - df_quiz_attempts['Quiz Startzeit']

df_quiz_grade = pd.DataFrame(quiz_grade_data)

#df_course_modules = pd.DataFrame(course_modules_data)

df_course = pd.DataFrame(course_data)

df_user_enrolments = pd.DataFrame(user_enrolments_data)

df_enrol = pd.DataFrame(enrol_data)

df_course_sections = pd.DataFrame(course_sections_data)

df_quiz = pd.DataFrame(quiz_data)

df = pd.merge(df_user, df_quiz_attempts, on = 'SuS ID', how = 'left')
df = pd.merge(df, df_quiz_grade, on = ['SuS ID', 'Quiz ID'], how = 'left')
df = pd.merge(df, df_user_enrolments, on = 'SuS ID', how = 'left')
df = pd.merge(df, df_enrol, on = 'Anmelde ID', how = 'left')
df = pd.merge(df, df_course, on = 'Klassen ID', how = 'left')
df = pd.merge(df, df_course_sections, on = 'Klassen ID', how = 'left')
df = pd.merge(df, df_quiz, on = 'Quiz ID', how = 'left')
df['Beste Punktzahl in %'] = df['Beste Punktzahl in %']*10
df['Maximale Note in %'] = df['Maximale Note in %']*10
df = df.drop([0,1,2,3,4,5,6,7])
df['Alle Fächer'] = 'Alle Fächer'
df = df.drop_duplicates()

# Ab hier kann der Code weiter ausgeführt werden, falls der Datensatz aus der csv-Datei ausgelesen wurde


<div class="alert alert-block alert-info">
<b>Wichtige Listen</b>
</div>

In [ ]:
subject_list = course_sections['name'].unique().tolist()
subject_list.remove(None)
subject_list.remove('Organisation')
class_list = df['Klasse'].unique().tolist()
exercise_list = quiz['name'].unique().tolist()
exercise_list.remove('Auswahl der übermittelten Daten')
name_list = df['Name'].unique().tolist()
name_list.sort()
class_list.sort()

# add general overview over all exercises
default_exercise = ['Allgemein']
df[default_exercise] = default_exercise

class_list_with_none = ['Keine Auswahl'] + class_list
name_list_with_none = ['Keine Auswahl'] + name_list
subject_list_with_all = ['Alle Fächer'] + subject_list

df['Einwilligung abgegeben'] = np.where(df['Aufgabe'] == 'Auswahl der übermittelten Daten', 'Ja', 'Nein')
consent_list = df.loc[df['Einwilligung abgegeben'] == 'Ja', 'Name']
df['Einwilligung abgegeben'] = np.where(df['Name'].isin(consent_list), 'Ja', 'Nein')

# Datensouveränität

<div class="alert alert-block alert-info">
<b>Erstellung des benötigten Datensatzes und Herauslesen der zu löschenden Daten</b>
</div>

In [ ]:
df_answers = pd.DataFrame(question_attempts_data)
df_answers_names = pd.DataFrame(question_data)
df_souv = df[df['Aufgabe'] == 'Auswahl der übermittelten Daten']

df_souv = pd.merge(df_souv, df_answers, on = 'Fragennutzung ID', how = 'left')
df_souv = pd.merge(df_souv, df_answers_names, on = 'Frage ID', how = 'left')
student_ids = df_souv['SuS ID'].unique().tolist()
df_souv = df_souv[df_souv.Fach != 'Mathematik']
df_souv = df_souv.reset_index()
df_souv = df_souv.drop_duplicates()

<div class="alert alert-block alert-danger">
<b>Ab hier die Zellen ausführen, wenn keine Verbindung zur MariaDB besteht.</b>
</div>

In [ ]:
df = pd.read_csv('moodle_dataset.csv')
df['Quiz Startzeit'] = pd.to_datetime(df['Quiz Startzeit'], format="%Y-%m-%d %H:%M:%S")
df['Quiz Abgabezeit'] = pd.to_datetime(df['Quiz Abgabezeit'], format="%Y-%m-%d %H:%M:%S")
df['Quiz Bearbeitungsdauer'] = df['Quiz Abgabezeit'] - df['Quiz Startzeit']
df['Einwilligung abgegeben'] = np.where(df['Aufgabe'] == 'Auswahl der übermittelten Daten', 'Ja', 'Nein')
consent_list = df.loc[df['Einwilligung abgegeben'] == 'Ja', 'Name']
df['Einwilligung abgegeben'] = np.where(df['Name'].isin(consent_list), 'Ja', 'Nein')

subject_list = df['Fach'].unique().tolist()

df_souv = pd.read_csv('df_souv.csv')
df_souv['Quiz Startzeit'] = pd.to_datetime(df_souv['Quiz Startzeit'], format="%Y-%m-%d %H:%M:%S")
df_souv['Quiz Abgabezeit'] = pd.to_datetime(df_souv['Quiz Abgabezeit'], format="%Y-%m-%d %H:%M:%S")
df_souv['Quiz Bearbeitungsdauer'] = df_souv['Quiz Abgabezeit'] - df['Quiz Startzeit']
student_ids = df_souv['SuS ID'].unique().tolist()

<div class="alert alert-block alert-info">
<b>Hilfsfunktion zum Löschen von Daten</b>
</div>

In [ ]:
# student_id: student
# start_time: start of period in which data needs to be deleted
# end_time: end of period that needs to be deleted
# question: column in which data needs to be deleted (will be replaced with NaN)
# index_list: row index in which data needs to be deleted (will be replaced with NaN)

def delete_data(student_id, start_time, end_time, question, index_list):
    # no end of period
    if end_time == 0:
        for index in index_list:
            if ((df.at[index,'Quiz Abgabezeit'] >= start_time) or pd.isna(df.at[index,'Quiz Abgabezeit'])):
                df.at[index, question] = np.NaN
                
                # also delete marks in percent
                if question == 'Punktzahl':
                    df.at[index, 'Beste Punktzahl in %'] = np.NaN
                    df.at[index, 'Maximale Punktzahl'] = np.NaN
                    df.at[index, 'Maximale Note in %'] = np.NaN
                
                if (question == 'Quiz Startzeit') or (question == 'Quiz Abgabezeit'):
                    df.at[index, 'Quiz Bearbeitungsdauer'] = np.NaN
    
    # end of period of time exists thus only delete necessary data in this period of time
    else:
        for index in index_list:
            if ((df.at[index,'Quiz Abgabezeit'] >= start_time and df.at[index,'Quiz Abgabezeit'] <= end_time) or pd.isna(df.at[index,'Quiz Abgabezeit'])):
                df.at[index, question] = np.NaN
                
                if question == 'Punktzahl':
                    df.at[index, 'Beste Punktzahl in %'] = np.NaN

In [ ]:
for student_id in student_ids:
    times = df_souv['Quiz Startzeit'].unique().tolist()
    question_list = df_souv['Frage Name'].unique().tolist()
    index_list = df.index[df['SuS ID'] == student_id].tolist()

    if len(times) >= 1:
        t = 0
        for time in times:
            for question in question_list:
                if(t == len(times)-1):
                    break
                    
                i = 0
                while i < len(question_list):
                    if df_souv.at[i, 'Frage Name'] == question and df_souv.at[i, 'Antwort'] == 'Falsch':
                        delete_data(student_id, time, times[t+1], question, index_list)
                    i += 1
            t += 1
        
        for question in question_list:
            
            i = (len(times)-1)*len(question_list)
            while i < len(question_list)*(len(times)):
                if df_souv.at[i, 'Frage Name'] == question and df_souv.at[i, 'Antwort'] == 'Falsch':
                    delete_data(student_id, times[-1], 0, question, index_list)
                i += 1
                
    else:
        for time in times:
            for question in question_list:
                i = 0
                while i < len(question_list)*len(times):
                    if df_souv.at[i, 'Frage Name'] == question and df_souv.at[i, 'Antwort'] == 'Falsch':
                        delete_data(student_id, time, 0, question, index_list)
                    i += 1

# Datensatz mit generierten Daten anreichern

In [ ]:
gen_data = pd.read_csv('extension_for_moodle_data.csv', quoting = QUOTE_NONE)

In [ ]:
gen_data['Quiz Startzeit'] = pd.to_datetime(gen_data['Quiz Startzeit'], format="%Y-%m-%d %H:%M:%S")
gen_data['Quiz Abgabezeit'] = pd.to_datetime(gen_data['Quiz Abgabezeit'], format="%Y-%m-%d %H:%M:%S")
gen_data['Quiz Bearbeitungsdauer'] = gen_data['Quiz Abgabezeit'] - gen_data['Quiz Startzeit']

In [ ]:
df = pd.concat([df, gen_data], ignore_index = True)

In [ ]:
# get good data types
df['Punktzahl'] = df['Punktzahl'].apply(float)
df['Quiz ID'] = df['Quiz ID'].fillna(0).apply(int)

<div class="alert alert-block alert-info">
<b>Listen neu anlegen</b>
</div>

In [ ]:
subject_list = df['Fach'].unique().tolist()
subject_list.remove('Organisation')

class_list = df['Klasse'].unique().tolist()

exercise_list = df['Aufgabe'].unique().tolist()
exercise_list.remove('Auswahl der übermittelten Daten')

name_list = df['Name'].unique().tolist()

name_list = [n for n in name_list if str(n) != 'nan']
class_list = [c for c in class_list if str(c) != 'nan']
subject_list.remove(np.nan)
exercise_list.remove(np.nan)

name_list.sort()
class_list.sort()

# add general overview over all exercises
default_exercise = ['Allgemein']
df[default_exercise] = default_exercise

class_list_with_none = ['Keine Auswahl'] + class_list
name_list_with_none = ['Keine Auswahl'] + name_list
subject_list_with_all = ['Alle Fächer'] + subject_list

# Funktionalitäten

<div class="alert alert-block alert-info">
<b>Buttons und Menüs</b>
</div>

In [ ]:
exercise_button = pn.widgets.RadioButtonGroup(
    name='exercise_button', options = default_exercise + exercise_list, button_type='success', orientation = 'vertical')

subject_button = pn.widgets.RadioButtonGroup(
    name='Fach auswählen', options = subject_list_with_all, button_type='success')

class_selection = pn.widgets.Select(name='Klasse auswählen', options = class_list_with_none)
student_selection = pn.widgets.Select(name='Schüler*in auswählen', options = name_list_with_none)

start_date_picker = pn.widgets.DatetimePicker(name='Startdatum auswählen', value=dt.datetime(2023, 1, 1, 0, 0))
end_date_picker = pn.widgets.DatetimePicker(name='Enddatum auswählen', value=dt.datetime(2023, 12, 31, 0, 0))

# Graphen

<div class="alert alert-block alert-info">
<b>Wichtige Spalten hinzufügen</b>
</div>

In [ ]:
df['Anzahl'] = df.groupby(['Punktzahl'])['Klasse'].transform('count')
df['Durchschnitt Punktzahl'] = df.groupby(['Aufgabe'])['Punktzahl'].transform('mean')
df['Median Punktzahl'] = df.groupby(['Aufgabe'])['Punktzahl'].transform('median')
df['Durchschnitt Fehlversuche'] = df.groupby(['Aufgabe'])['Anzahl Versuche'].transform('mean')
df['Median Fehlversuche'] = df.groupby(['Aufgabe'])['Anzahl Versuche'].transform('median')

df['Median Bearbeitungsdauer'] = df.groupby(['Aufgabe', 'Name', 'Fach'])['Quiz Bearbeitungsdauer'].transform('median')
df['Median Bearbeitungsdauer'] = df['Median Bearbeitungsdauer'].apply(
    lambda x: f'{x.components.hours:02d}:{x.components.minutes:02d}:{x.components.seconds:02d}'
              if not pd.isnull(x) else ''
)

df['Anzahl Aufgaben pro Klasse'] = df['Klassen ID']

# auskommentierte Zeile funktioniert nur mit der MariaDB
# quiz['Anzahl Aufgaben pro Klasse'] = quiz.groupby(['course'])['id'].transform('count')
quiz = pd.read_csv('quiz.csv')
quiz['Anzahl Aufgaben pro Klasse'] = quiz.groupby(['course'])['id'].transform('count')

aufgaben_dict = dict(zip(quiz['course'], quiz['Anzahl Aufgaben pro Klasse']))
aufgaben_dict[5] = 0
df['Anzahl Aufgaben pro Klasse'] = df['Anzahl Aufgaben pro Klasse'].map(aufgaben_dict).fillna(df['Anzahl Aufgaben pro Klasse'])

df['Anzahl erledigter Aufgaben'] = df['Name']
df_temp = df[['Name', 'Quiz ID']]
df_temp = df_temp[df_temp['Quiz ID'] != 0]
df_temp = df_temp[df_temp['Name'] != 'Alina Neue']
df_temp = df_temp.drop_duplicates()
df_temp['Anzahl erledigter Aufgaben'] = df_temp.groupby(['Name']).transform('count')
aufgaben_dict = dict(zip(df_temp['Name'], df_temp['Anzahl erledigter Aufgaben']))
df['Anzahl erledigter Aufgaben'] = df['Anzahl erledigter Aufgaben'].map(aufgaben_dict).fillna(df['Anzahl erledigter Aufgaben'])
df['Anzahl erledigter Aufgaben'] = pd.to_numeric(df['Anzahl erledigter Aufgaben'], errors = 'coerce')
df['Anzahl erledigter Aufgaben'] = df['Anzahl erledigter Aufgaben'].fillna(0)

df['Lernfortschritt'] = df['Anzahl erledigter Aufgaben']/df['Anzahl Aufgaben pro Klasse']*100
df['Lernfortschritt'] = df['Lernfortschritt'].fillna(0).apply(float)

df['Anzahl Versuche'] = df.groupby(['Aufgabe', 'Name', 'Fach'])['Anzahl Versuche'].transform('max')

df['Einwilligung abgegeben'] = df['Einwilligung abgegeben'].replace(np.nan, 'Nein')

df['Quiz Abgabedatum'] = df['Quiz Abgabezeit'].dt.date

df['Durchschnitt Punktzahl Aufgabe'] = df.groupby(['Aufgabe'])['Punktzahl'].transform('mean')
df['Durchschnitt Punktzahl Schüler*in'] = df.groupby(['Aufgabe', 'Name'])['Punktzahl'].transform('mean')

df['Durchschnitt Bearbeitungsdauer Aufgabe'] = df.groupby(['Aufgabe'])['Quiz Bearbeitungsdauer'].transform('mean')
df['Durchschnitt Bearbeitungsdauer Aufgabe'] = df['Durchschnitt Bearbeitungsdauer Aufgabe'].apply(
    lambda x: f'{x.components.hours:02d}:{x.components.minutes:02d}:{x.components.seconds:02d}'
              if not pd.isnull(x) else ''
)

df['Durchschnitt Bearbeitungsdauer Schüler*in'] = df.groupby(['Aufgabe', 'Name'])['Quiz Bearbeitungsdauer'].transform('mean')
df['Durchschnitt Bearbeitungsdauer Schüler*in'] = df['Durchschnitt Bearbeitungsdauer Schüler*in'].apply(
    lambda x: f'{x.components.hours:02d}:{x.components.minutes:02d}:{x.components.seconds:02d}'
              if not pd.isnull(x) else ''
)

df['Durchschnitt Fehlversuche Aufgabe'] = df.groupby(['Aufgabe'])['Anzahl Versuche'].transform('mean')
df['Durchschnitt Fehlversuche Schüler*in'] = df.groupby(['Aufgabe', 'Name'])['Anzahl Versuche'].transform('mean')

df['Punktzahl im Vergleich in %'] = (df['Durchschnitt Punktzahl Schüler*in']/df['Durchschnitt Punktzahl Aufgabe'])*100
df['Fehlversuche im Vergleich in %'] = (df['Durchschnitt Fehlversuche Schüler*in']/df['Durchschnitt Fehlversuche Aufgabe'])*100

# Funktionen für Tabellen

<div class="alert alert-block alert-info">
<b>Aufgaben: Anzahl der Punktzahlen</b>
</div>

In [ ]:
count_int = df.interactive()

count_pipeline = (
    count_int[
        ((count_int['Fach'] == subject_button) | (count_int['Alle Fächer'] == subject_button)) &
        ((count_int['Quiz Abgabezeit'] >= start_date_picker) & (count_int['Quiz Abgabezeit'] <= end_date_picker)) &
        ((count_int['Aufgabe'] == exercise_button) | (count_int['Allgemein'] == exercise_button)) &
        ((count_int['Klasse'] == class_selection) | (count_int['Name'] == student_selection))
    ]
    .groupby(['Punktzahl'])['Anzahl'].count()
    .to_frame()
    .reset_index()
    .reset_index(drop=True)
)

In [ ]:
count_plot = count_pipeline.hvplot.scatter(x='Punktzahl', y='Anzahl', title='Anzahl der Punktzahl', xlabel = 'Punktzahl', ylabel = 'Anzahl Schüler*innen').output()

<div class="alert alert-block alert-info">
<b>Aufgaben: Durchschnitte der Aufgaben</b>
</div>

In [ ]:
mean_int = df.interactive()

mean_pipeline = (
    mean_int[
        ((mean_int['Fach'] == subject_button) | (mean_int['Alle Fächer'] == subject_button)) &
        ((mean_int['Quiz Startzeit'] >= start_date_picker) & (mean_int['Quiz Abgabezeit'] <= end_date_picker)) &
        ((mean_int['Aufgabe'] == exercise_button) | (mean_int['Allgemein'] == exercise_button)) &
        ((mean_int['Klasse'] == class_selection) | (mean_int['Name'] == student_selection))
    ]
    .groupby(['Aufgabe', 'Durchschnitt Fehlversuche Aufgabe', 'Durchschnitt Bearbeitungsdauer Aufgabe'])['Durchschnitt Punktzahl Aufgabe'].mean()
    .to_frame()
    .reset_index()
)

mean_table = mean_pipeline.pipe(pn.widgets.Tabulator, pagination = 'remote', page_size = 10, theme = 'fast', hidden_columns = ['index']).output()

<div class="alert alert-block alert-info">
<b>Aufgaben: Durchschnitt vs Median Punktzahl</b>
</div>

In [ ]:
median_int = df.interactive()

median_pipeline = (
    median_int[
        ((median_int['Fach'] == subject_button) | (median_int['Alle Fächer'] == subject_button)) &
        ((median_int['Quiz Startzeit'] >= start_date_picker) & (median_int['Quiz Abgabezeit'] <= end_date_picker)) &
        ((median_int['Aufgabe'] == exercise_button) | (median_int['Allgemein'] == exercise_button)) &
        ((median_int['Klasse'] == class_selection) | (median_int['Name'] == student_selection))
    ]
    .groupby(['Aufgabe', 'Durchschnitt Punktzahl Aufgabe'])['Median Punktzahl'].median()
    .to_frame()
    .reset_index()
)

median_punktzahl_plot = median_pipeline.hvplot.bar(x='Aufgabe', title='Durchschnitt und Median der Punktzahl', xlabel = 'Aufgabe', ylabel = 'Punktzahl', rot=45).output()
median_punktzahl_plot = median_punktzahl_plot.opts(multi_level=False)

<div class="alert alert-block alert-info">
<b>Schüler*in: Durchschnitt</b>
</div>

In [ ]:
student_int = df.interactive()

student_pipeline = (
    student_int[
        ((student_int['Fach'] == subject_button) | (student_int['Alle Fächer'] == subject_button)) &
        ((student_int['Quiz Startzeit'] >= start_date_picker) & (student_int['Quiz Abgabezeit'] <= end_date_picker)) &
        ((student_int['Aufgabe'] == exercise_button) | (student_int['Allgemein'] == exercise_button)) &
        ((student_int['Klasse'] == class_selection) | (student_int['Name'] == student_selection))
    ]
    .reset_index()
)

student_table = student_pipeline[['Name', 'Aufgabe', 'Durchschnitt Punktzahl Schüler*in', 'Punktzahl im Vergleich in %', 'Durchschnitt Fehlversuche Schüler*in', 'Fehlversuche im Vergleich in %', 'Durchschnitt Bearbeitungsdauer Schüler*in']].drop_duplicates().pipe(pn.widgets.Tabulator, pagination = 'remote', page_size = 10, theme = 'fast', hidden_columns = ['index']).output()

<div class="alert alert-block alert-info">
<b>Lernfortschritt</b>
</div>

In [ ]:
count_exercise_int = df.interactive()

count_exercise_pipeline = (
    count_exercise_int[
        ((count_exercise_int['Fach'] == subject_button) | (count_exercise_int['Alle Fächer'] == subject_button)) &
        ((count_exercise_int['Quiz Startzeit'] >= start_date_picker) & (count_exercise_int['Quiz Abgabezeit'] <= end_date_picker)) &
        ((count_exercise_int['Aufgabe'] == exercise_button) | (count_exercise_int['Allgemein'] == exercise_button)) &
        ((count_exercise_int['Klasse'] == class_selection) | (count_exercise_int['Name'] == student_selection))        
    ]
    .groupby(['Name', 'Anzahl erledigter Aufgaben'])['Lernfortschritt'].mean()
    .to_frame()
    .reset_index()
)

count_exercise_table = count_exercise_pipeline.pipe(pn.widgets.Tabulator, pagination = 'remote', page_size = 10, theme = 'fast', hidden_columns = ['index']).output()

<div class="alert alert-block alert-info">
<b>Leistung über Zeit</b>
</div>

In [ ]:
time_scores_int = df.interactive()

time_scores_pipeline = (
    time_scores_int[
        ((time_scores_int['Fach'] == subject_button) | (time_scores_int['Alle Fächer'] == subject_button)) &
        ((time_scores_int['Quiz Startzeit'] >= start_date_picker) & (time_scores_int['Quiz Abgabezeit'] <= end_date_picker)) &
        ((time_scores_int['Aufgabe'] == exercise_button) | (time_scores_int['Allgemein'] == exercise_button)) &
        ((time_scores_int['Klasse'] == class_selection) | (time_scores_int['Name'] == student_selection))        
    ]
    .pivot_table(index = 'Name', columns = 'Quiz Abgabedatum', values = 'Punktzahl', aggfunc = 'mean', fill_value = np.nan) 
)

time_scores_table = time_scores_pipeline.pipe(pn.widgets.Tabulator, pagination = 'remote', page_size = 10, theme = 'fast', hidden_columns = ['index']).output()

<div class="alert alert-block alert-info">
<b>Schüler*innen, welche noch keine Einvverständniserklärung abgegeben haben</b>
</div>

<div class="alert alert-block alert-warning">
<b>Zeigt bei Auswahl einzelner Schüler*innen nichts an.</b>
</div>

In [ ]:
no_consent_int = df.interactive()

no_consent_pipeline = (
    no_consent_int[
        ((no_consent_int['Fach'] == subject_button) | (no_consent_int['Alle Fächer'] == subject_button)) &
        ((no_consent_int['Quiz Startzeit'] >= start_date_picker) & (no_consent_int['Quiz Abgabezeit'] <= end_date_picker)) &
        ((no_consent_int['Aufgabe'] == exercise_button) | (no_consent_int['Allgemein'] == exercise_button)) &
        ((no_consent_int['Klasse'] == class_selection) | (no_consent_int['Name'] == student_selection)) &
        ((no_consent_int['Einwilligung abgegeben'] == 'Nein'))
    ]
)

no_consent_table = no_consent_pipeline[['Name', 'Einwilligung abgegeben']].drop_duplicates().pipe(pn.widgets.Tabulator, pagination = 'remote', page_size = 10, theme = 'fast', hidden_columns = ['index']).output()

# Create Teacher Dashboard

In [ ]:
#Layout using Template
template = pn.template.FastListTemplate(
    title='Learning Analytics Dashboard', 
    sidebar=[pn.pane.Markdown("# Auswahl"),  
             class_selection,
             student_selection,
             exercise_button],
    main=[pn.Row(pn.Column(subject_button),
                 pn.Column(start_date_picker, height = 100), 
                 pn.Column(end_date_picker)),
          pn.Row(pn.Column(median_punktzahl_plot), 
                    pn.Column(time_scores_table, width = 1000), height = 250),
          pn.Row(pn.Column(count_exercise_table), 
                    pn.Column(mean_table), 
                 pn.Column(no_consent_table), height = 250),
          pn.Row(pn.Column(student_table), height = 250),

     ],
    accent = '#88d8b0'
)

template.show()

#template.servable();
#command: panel serve --port 53369 learning_analytics_dashboard.ipynb in terminal then click on link